# Efficient rank training

Let's start with setting up the notebook. Skip it in case you have installed fedcore via pip and are able to import it

In [22]:
import os 
os.getcwd()

'/ptls-experiments/FedCore/examples'

In [23]:
os.chdir('../..')

In [24]:
# Older CUDA's version may not support expandable objects in CUDA malloc. This will fix it
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = ""

## Congiguration

In [25]:
from fedcore.tools.example_utils import get_scenario_for_api
from fedcore.api.main import FedCore
from fedcore.api.utils.checkers_collection import ApiConfigCheck
from fedcore.data.dataloader import load_data
from fedcore.api.utils.evaluation import evaluate_original_model, evaluate_optimised_model
from fedcore.repository.config_repository import DEFAULT_CLF_API_CONFIG

DATASET = 'CIFAR10'
DATASET_PARAMS = {'train_bs': 64,
                  'val_bs': 100,
                  'train_shuffle': True,
                  'val_shuffle': False}
METRIC_TO_OPTIMISE = ['accuracy', 'latency', 'throughput']
initial_assumption = 'ResNet18'
initial_assumption, learning_strategy = get_scenario_for_api('checkpoint', initial_assumption)

USER_CONFIG = {'problem': 'classification',
               'metric': METRIC_TO_OPTIMISE,
               'initial_assumption': initial_assumption,
               'pop_size': 1, # how many models to train in parallel
               'timeout': 1.3, # how long optimization process runs (in minutes)
               'learning_strategy': 'from_checkpoint', # whether we have pretrained model or not
               'learning_strategy_params': dict(epochs=0,
                                                learning_rate=0.0001,
                                                loss='crossentropy',
                                                custom_loss = [], # needs to rework BaseNN class
                                                custom_learning_params=dict(use_early_stopping={'patience': 30,
                                                                                                'maximise_task': False,
                                                                                                'delta': 0.01})
                                                ), # activates if basic pretrain is needed
               'peft_strategy': 'low_rank', # the compression approach
               'peft_strategy_params': dict(
                   log_each=1, # how often to print train losses
                   eval_each=5, # how often to evaluate model on validation dataset
                   scheduler='one_cycle', # which lr scheduler to use
                   epochs=10,
                   loss='crossentropy',
                   non_adaptive_threshold=0.2, # threshold for S-strategies
                    custom_criterions = {'hoer': 0.5, 'orthogonal': 0.2}, # Addtional losses with their coefficients
                   finetune_params={'epochs': 1,
                                    "learning_rate": 0.0001,
                                    'loss': 'crossentropy'} # parameters for finetuning model after rank pruning
                ),
               }

# Initialization of API config
api_config = ApiConfigCheck().update_config_with_kwargs(DEFAULT_CLF_API_CONFIG, **USER_CONFIG)

## Start learning

In [26]:
# use built-in dataset loader or define your own
input_data = load_data(DATASET)
# initialize compressor instance
fedcore_compressor = FedCore(**api_config)

# if we have already got the compression pipeline, we may launch fitting without evolutionary search
x = fedcore_compressor.fit_no_evo(input_data)

# else we may run it like this:
# fedcore_compressor.fit(input_data);

Files already downloaded and verified
Files already downloaded and verified
Creating Dask Server
Triggered OptimizerGen at 1 epoch.
Triggered SchedulerRenewal at 1 epoch.


Batch #: 100%|██████████| 782/782 [00:45<00:00, 17.30it/s]

Triggered Evaluator at 1 epoch.



Batch #: 100%|██████████| 100/100 [00:01<00:00, 63.36it/s]

Triggered FitReport at 1 epoch.
Train # epoch: 1, value: 7.273792279041027
Valid # epoch: 1, value: 6.7633562088012695
Including:
	Criterion `train_hoer_loss`: 5.835456371307373
	Criterion `val_hoer_loss`: 5.83475923538208
	Criterion `train_orthogonal_loss`: 0.0013179627712816
	Criterion `val_orthogonal_loss`: 0.0013199361274018884


...


Batch #: 100%|██████████| 100/100 [00:01<00:00, 57.53it/s]

Triggered FitReport at 10 epoch.
Train # epoch: 10, value: 2.729780821849013
Valid # epoch: 10, value: 2.838242530822754
Including:
	Criterion `train_hoer_loss`: 2.0968384742736816
	Criterion `val_hoer_loss`: 2.096817970275879
	Criterion `train_orthogonal_loss`: 0.03219258412718773
	Criterion `val_orthogonal_loss`: 0.03218986466526985


==============Truncate rank for each weight matrix=================
After rank pruning left only 100.0 % of conv1 layer params
After rank pruning left only 100.0 % of layer1.0.conv1 layer params
After rank pruning left only 100.0 % of layer1.0.conv2 layer params
After rank pruning left only 100.0 % of layer1.1.conv1 layer params
After rank pruning left only 100.0 % of layer1.1.conv2 layer params
After rank pruning left only 50.0 % of layer2.0.conv1 layer params
After rank pruning left only 50.0 % of layer2.0.conv2 layer params
After rank pruning left only 100.0 % of layer2.0.downsample.0 layer params
After rank pruning left only 50.0 % of layer2.1.conv1 layer params
After rank pruning left only 50.0 % of layer2.1.conv2 layer params
After rank pruning left only 75.0 % of layer3.0.conv1 layer params
After rank pruning left only 75.0 % of layer3.0.conv2 layer params
After rank pruning left only 50.0 % of layer3.0.downsample.0 layer params
After rank pruning left only 75.0 % of layer3.1.co

Batch #: 100%|██████████| 782/782 [00:21<00:00, 36.98it/s]


Triggered FitReport at 1 epoch.
Train # epoch: 1, value: 1.4241080399974229
Valid # epoch: 10, value: 2.838242530822754
Including:
	Criterion `train_hoer_loss`: 0.0
	Criterion `val_hoer_loss`: 2.096817970275879
	Criterion `train_orthogonal_loss`: 0.28798818588256836
	Criterion `val_orthogonal_loss`: 0.03218986466526985
Triggered FitReport at 1 epoch.
Train # epoch: 1, value: 1.4241080399974229
Valid # epoch: 10, value: 2.838242530822754
Including:
	Criterion `train_hoer_loss`: 0.0
	Criterion `val_hoer_loss`: 2.096817970275879
	Criterion `train_orthogonal_loss`: 0.28798818588256836
	Criterion `val_orthogonal_loss`: 0.03218986466526985
==============After low rank truncation=================
Params: 13.44 M => 11.69 M
MACs: 0.00 G => 0.00 G


In [27]:
opt_model = fedcore_compressor.optimised_model.model